In [17]:
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import os


DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
PRODUCTION_SCHEMA = os.getenv("PRODUCTION_SCHEMA")

pd.options.plotting.backend = "plotly"



def get_engine_connection():
        """Connects to postgreSQL DBMS on AWS Aurora

        Returns:
            DB engine
        """
        conn_string = (
            f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
        )

        return create_engine(conn_string)

In [18]:
query = f"""
    WITH user_gender_dob AS (
        SELECT user_id, gender,
            DATE_PART('year', AGE(CURRENT_DATE, date_of_birth))
                AS age 
            FROM {PRODUCTION_SCHEMA}.users
    ),

    rides_before AS (
        SELECT *
            FROM {PRODUCTION_SCHEMA}.rides
            WHERE begin_timestamp > (CURRENT_DATE - INTERVAL '24 hours')
    )

    SELECT ugd.user_id, rb.ride_id, ugd.gender, ugd.age, rb.begin_timestamp,
        rb.total_duration_sec, rb.total_power, rb.mean_power, rb.mean_resistance,
            rb.mean_rpm, rb.mean_heart_rate

        FROM user_gender_dob AS ugd
        RIGHT JOIN rides_before AS rb
            ON ugd.user_id = rb.user_id
"""

df = pd.read_sql(query, con=get_engine_connection())

In [19]:
df

,user_id,ride_id,gender,age,begin_timestamp,total_duration_sec,total_power,mean_power,mean_resistance,mean_rpm,mean_heart_rate
0,4694,242,Female,48.0,2022-10-09 00:07:59,520,36341.015,69.886567,69.600000,46.780769,107.096154
1,4694,243,Female,48.0,2022-10-09 00:16:41,520,33516.201,64.454233,42.200000,50.990385,123.903846
2,4695,244,Male,45.0,2022-10-09 00:25:24,520,26853.197,51.640763,40.253846,49.525000,143.648077
3,4695,245,Male,45.0,2022-10-09 00:34:07,501,20361.827,40.642369,40.546906,46.670659,107.680639
4,4695,246,Male,45.0,2022-10-09 00:42:30,519,22345.647,43.055197,40.100193,39.142582,97.988439
...,...,...,...,...,...,...,...,...,...,...,...
253,4754,499,Female,49.0,2022-10-10 12:27:16,518,21258.790,41.040135,39.814672,50.397683,110.291506
254,4754,500,Female,49.0,2022-10-10 12:35:57,520,21853.773,42.026487,39.769231,41.805769,118.005769
255,4754,501,Female,49.0,2022-10-10 12:44:40,518,23816.278,45.977371,39.795367,52.330116,117.774131
256,4754,502,Female,49.0,2022-10-10 12:53:20,514,19374.631,37.693835,39.976654,39.992218,81.268482


### Daily Report Requirements

* Number of rides completed in the past day

* Gender split of riders of the past day

* Ages of the riders of the past day

* Average power and heart rate of riders of the past day